<a href="https://colab.research.google.com/github/chamisfum/Brain_Tumor_Classification_Trial/blob/master/Model_Percobaan3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import numpy as np
import os
import shutil
from tqdm import tqdm
import cv2
import matplotlib.pyplot as plt

%matplotlib inline

In [9]:
os.environ['CONFIG_DIR'] = "/content/drive/My Drive/RESEARCH CENTER/DATASETS/BRAIN_TUMOR/dataset"

In [10]:
%cd /content/drive/My Drive/RESEARCH CENTER/DATASETS/BRAIN_TUMOR/dataset

/content/drive/My Drive/RESEARCH CENTER/DATASETS/BRAIN_TUMOR/dataset


In [11]:
ls

AUGMENTATION/  draft/      percobaan/  save_model/
BALANCE/       IMBALANCE/  readme.txt  SPLIT_AUGMENTATION/


In [12]:
ls percobaan

percobaan1.png  percobaan2.png  percobaan3.png  test/  train/  val/


In [13]:
import tensorflow as tf
from tensorflow.keras import applications
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras import callbacks
from tensorflow.keras import optimizers

In [ ]:
# model = Sequential()
# model.add(base_model)
# model.add(Flatten())
# model.add(Dense(300,activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.005)))
# model.add(Dense(3,activation='softmax'))
# model.layers[0].trainable = False

# lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
#     initial_learning_rate=0.01,
#     decay_steps=500,
#     decay_rate=0.0001)
# opt = tf.keras.optimizers.Adamax(learning_rate=lr_schedule)

# model.compile(
#     loss='categorical_crossentropy',
#     optimizer=opt,
#     metrics=['accuracy']
# )

# model.summary()

In [14]:
path_train = 'percobaan/train/'
path_val = 'percobaan/val/'
path_test = 'percobaan/test/'

In [16]:
from keras.preprocessing.image import ImageDataGenerator


datagen = ImageDataGenerator(rescale=1./255,
                            #  rotation_range=40,
                            #  width_shift_range=0.2,
                            #  height_shift_range=0.2,
                            #  shear_range=0.2,
                            #  zoom_range=0.2,
                            #  horizontal_flip=True,
                             fill_mode='nearest')

training_set = datagen.flow_from_directory(path_train,
                                           target_size=(128,128),
                                           color_mode="rgb",
                                           shuffle=False,
                                           batch_size = 64,
                                           class_mode='categorical')

validation_set = datagen.flow_from_directory(path_val,
                                             target_size=(128,128),
                                             color_mode="rgb",
                                             shuffle=False,
                                             batch_size = 64,
                                             class_mode='categorical')

testing_set = datagen.flow_from_directory(path_test,
                                          target_size=(128,128),
                                          color_mode="rgb",
                                          shuffle=False,
                                          batch_size = 64,
                                          class_mode='categorical')

Found 12256 images belonging to 3 classes.
Found 2297 images belonging to 3 classes.
Found 767 images belonging to 3 classes.


In [17]:
# Found 12256 images belonging to 3 classes.
# Found 2297 images belonging to 3 classes.
# Found 767 images belonging to 3 classes.

In [18]:
training_set.class_indices

{'GLIOMA': 0, 'MENINGIOMA': 1, 'PITUITARY': 2}

In [ ]:
base_model = applications.MobileNetV2(include_top=False,weights='imagenet',input_shape=(128,128,3))
base_model.summary()

Model: "mobilenetv2_1.00_128"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 129, 129, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
_______________________________________________________________________________

In [ ]:
model = Sequential()
model.add(base_model)
model.add(layers.Flatten())
model.add(layers.Dense(1028, activation = 'relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(512, activation = 'relu'))
model.add(layers.Dense(256, activation = 'relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(3, activation = 'softmax'))
model.layers[0].trainable = False
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_128 (Functi (None, 4, 4, 1280)        2257984   
_________________________________________________________________
flatten_4 (Flatten)          (None, 20480)             0         
_________________________________________________________________
dense_17 (Dense)             (None, 1028)              21054468  
_________________________________________________________________
dropout_8 (Dropout)          (None, 1028)              0         
_________________________________________________________________
dense_18 (Dense)             (None, 512)               526848    
_________________________________________________________________
dense_19 (Dense)             (None, 256)               131328    
_________________________________________________________________
dropout_9 (Dropout)          (None, 256)              

In [21]:
from tensorflow.keras import models
model = models.load_model("/content/drive/My Drive/RESEARCH CENTER/DATASETS/BRAIN_TUMOR/dataset/save_model/temp_trial/049-val_acc:0.944.h5")
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_128 (Functi (None, 4, 4, 1280)        2257984   
_________________________________________________________________
flatten_4 (Flatten)          (None, 20480)             0         
_________________________________________________________________
dense_17 (Dense)             (None, 1028)              21054468  
_________________________________________________________________
dropout_8 (Dropout)          (None, 1028)              0         
_________________________________________________________________
dense_18 (Dense)             (None, 512)               526848    
_________________________________________________________________
dense_19 (Dense)             (None, 256)               131328    
_________________________________________________________________
dropout_9 (Dropout)          (None, 256)              

In [22]:
model.compile(loss = 'categorical_crossentropy',
              optimizer = optimizers.Adamax(lr=1e-4),
              metrics = ['acc'])

In [25]:
model.evaluate(training_set)

192/192 [==============================] - 8911s 46s/step - loss: 0.0033 - acc: 0.9995


[0.003313478548079729, 0.9995104670524597]

In [24]:
model.evaluate(validation_set)

36/36 [==============================] - 1520s 42s/step - loss: 0.1929 - acc: 0.9451


[0.19289983808994293, 0.945145845413208]

In [23]:
model.evaluate(testing_set)

12/12 [==============================] - 471s 39s/step - loss: 0.1920 - acc: 0.9439


[0.19199052453041077, 0.9439374208450317]

In [ ]:
filepath="/content/drive/My Drive/RESEARCH CENTER/DATASETS/BRAIN_TUMOR/dataset/save_model/temp_trial/{epoch:03d}-val_acc:{val_acc:.3f}.h5"
checkpoint = callbacks.ModelCheckpoint(
                                       filepath,
                                       monitor='val_acc',
                                       verbose=1,
                                       save_best_only=True,
                                       mode='auto',
                                      #  restore_best_weights=True,
                                       )
# earlystop = callbacks.EarlyStopping(monitor='val_loss',
#                                     mode='auto',
#                                     patience=4,
#                                     restore_best_weights=True)

In [ ]:
train_step_size = training_set.n//training_set.batch_size
val_step_size = validation_set.n//validation_set.batch_size

In [ ]:
history = model.fit(training_set, 
                    steps_per_epoch=train_step_size, 
                    epochs=50,
                    validation_data=validation_set, 
                    validation_steps=val_step_size, 
                    callbacks=[checkpoint])

Epoch 1/50
191/191 [==============================] - ETA: 0s - loss: 0.9873 - acc: 0.5974
Epoch 00001: val_acc improved from -inf to 0.71205, saving model to /content/drive/My Drive/RESEARCH CENTER/DATASETS/BRAIN_TUMOR/dataset/save_model/temp_trial/001-val_acc:0.712.h5
191/191 [==============================] - 64s 336ms/step - loss: 0.9873 - acc: 0.5974 - val_loss: 0.6176 - val_acc: 0.7121
Epoch 2/50
191/191 [==============================] - ETA: 0s - loss: 0.6185 - acc: 0.7340
Epoch 00002: val_acc improved from 0.71205 to 0.80759, saving model to /content/drive/My Drive/RESEARCH CENTER/DATASETS/BRAIN_TUMOR/dataset/save_model/temp_trial/002-val_acc:0.808.h5
191/191 [==============================] - 64s 338ms/step - loss: 0.6185 - acc: 0.7340 - val_loss: 0.4775 - val_acc: 0.8076
Epoch 3/50
191/191 [==============================] - ETA: 0s - loss: 0.4974 - acc: 0.7927
Epoch 00003: val_acc improved from 0.80759 to 0.81339, saving model to /content/drive/My Drive/RESEARCH CENTER/DATAS

In [ ]:
model.evaluate(training_set)

192/192 [==============================] - 52s 271ms/step - loss: 0.0088 - acc: 0.9967


[0.008758462965488434, 0.9966546893119812]

In [ ]:
model.evaluate(validation_set)

36/36 [==============================] - 9s 252ms/step - loss: 0.2198 - acc: 0.9412


[0.21982474625110626, 0.9412276744842529]

In [ ]:
model.evaluate(testing_set)

12/12 [==============================] - 315s 26s/step - loss: 0.2559 - acc: 0.9400


[0.2559448480606079, 0.9400261044502258]

In [6]:
# plot model performance
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs_range = range(1, len(history.epoch) + 1)

plt.figure(figsize=(15,5))

plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Train Set')
plt.plot(epochs_range, val_acc, label='Val Set')
plt.legend(loc="best")
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Model Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Train Set')
plt.plot(epochs_range, val_loss, label='Val Set')
plt.legend(loc="best")
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Model Loss')

plt.tight_layout()
plt.show()

NameError: ignored

In [7]:
ls

drive/  sample_data/


In [ ]:
model.save('save_model/P/trial_v7_model.h5') 

NameError: ignored

In [ ]:
model.save_weights("save_model/P/trial_v7_weights.h5")

In [ ]:
ls save_model/P